In [1]:
import sys
import os

# Add the parent directory to the system path
sys.path.append(os.path.abspath(os.path.join('..')))
sys.path.append(os.path.abspath(os.path.join('../..')))

In [2]:
import numpy as np
import torch
from lpn_mrs.networks import LPN_cond

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
b = 10
x = torch.rand(b, 1, 512).to(device)
sigma = torch.rand(b,1).to(device)

In [4]:
model = LPN_cond(in_dim = 1, hidden_c = 1, hidden = 30, kernel = 3, beta = 10, alpha = 1e-6)
model = model.to(device)
output = model(x, sigma)
model.init_weights(0, 1)
print(output.shape)

init weights
torch.Size([10, 1, 512])


In [5]:
def test_convexity(net, x, sigma, device=device):
    #check convexity of the net numerically
    print('running a numerical convexity test...')
    n_trials = 100
    convexity = 0
    for trial in np.arange(n_trials):
        x1 = torch.rand(x.size()).to(device)
        x2 = torch.rand(x.size()).to(device)
        alpha = torch.rand(1).to(device)
    
        cvx_combo_of_input = net(alpha * x1 + (1-alpha)*x2, sigma)
        cvx_combo_of_output = alpha * net(x1, sigma) + (1-alpha)*net(x2, sigma)
    
        convexity += (cvx_combo_of_input.mean() <= cvx_combo_of_output.mean())
        if cvx_combo_of_input.mean() > cvx_combo_of_output.mean():
            print(cvx_combo_of_input.mean(), cvx_combo_of_output.mean())
    if(convexity == n_trials):
        flag = True
        print('Passed convexity test!')
    else:
        flag = False
        print('Failed convexity test!')
    return flag

In [6]:
test_convexity(model, x, sigma)

running a numerical convexity test...
Passed convexity test!


True